# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [188]:
import sqlite3
from urllib.response import addbase

import numpy as np
import pandas as pd
import os

pd.set_option('display.max_column', None)
pd.set_option('display.max_rows',200)

In [189]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')
table_list = pd.read_sql("select * from sqlite_master where type = 'table';", db)
table_list

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [190]:
# You may load the data from SQL table directly to the Pandas dataframe as
table_list = pd.read_sql("select * from sqlite_master where type = 'table';", db)
player_data = pd.read_sql("SELECT * FROM Player;", db)
matches =  pd.read_sql("SELECT * FROM Match;", db)
leagues =  pd.read_sql("SELECT * FROM League;", db)
leagues = leagues.set_index('id')
players_attributes = pd.read_sql("SELECT * FROM Player_Attributes;", db)

**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [192]:
players_180_190 = (player_data[player_data['height'].between(180, 190)]).shape[0] # Your code here
# (player_data[player_data['height'].between(180, 190)])['height'].describe()

In [193]:
assert(isinstance(players_180_190, int))

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [194]:
player_data['year_of_birth'] = pd.to_datetime(player_data.birthday).apply(lambda x: x.year)
player_data['month_of_birth'] = pd.to_datetime(player_data.birthday).apply(lambda x: x.month)
player_data['dow_of_birth'] = pd.to_datetime(player_data.birthday).apply(lambda x: x.weekday())
players_1980 = player_data[(player_data.year_of_birth == 1980)].shape[0]

In [195]:
assert(isinstance(players_1980, int))

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [196]:
highest_players = player_data.sort_values(by=['weight', 'player_name'], ascending=[False, True]) \
    .head(10)['player_name'].tolist()

In [197]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [198]:
ye = pd.value_counts(player_data[player_data['year_of_birth'].between(1980, 1990)].year_of_birth)
ye = ye.sort_index()
years_born_players = list(zip(ye.index.tolist(), ye.tolist()))
# [1988, 1989, 1987, 1986, 1990, 1984, 1985, 1983, 1982, 1981, 1980]

In [199]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [200]:
pld = player_data
pld['name_separ'] = pld['player_name'].apply(lambda x: x.split(' ')[0])
ad_height = pld[pld['name_separ'] == 'Adriano']['height']
adriano_mean, adriano_std = ad_height.mean(), ad_height.std()# Your code here
adriano_mean, adriano_std

(182.11800000000002, 5.361479480723788)

In [201]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [202]:
dow = dict(zip(range(7), ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']) )
pld['dow_of_bd_string'] = pld['dow_of_birth'].apply(lambda x: dow[x])
hist = pld['dow_of_bd_string'].value_counts()
hist.sort_values(inplace=True)
/print hist
dow_with_min_players_born = hist.index[0]

Sunday       1362
Saturday     1463
Wednesday    1607
Monday       1643
Thursday     1648
Tuesday      1652
Friday       1685
Name: dow_of_bd_string, dtype: int64


In [203]:
assert(isinstance(dow_with_min_players_born, str))

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [204]:
leg_hist = matches['league_id'].value_counts()
leg_hist = pd.concat([leg_hist, leagues['name']], axis=1)
leg_hist.sort_values(by=['league_id', 'name'], ascending=[False, True])
league_most_matches = leg_hist.iloc[0, leg_hist.columns.get_loc('name')]

In [205]:
assert(isinstance(league_most_matches, str))

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [206]:
player_col_mask = matches.columns.str.match('\w*player\_\d{1,2}')
player_col_index = matches.columns[player_col_mask]
matches[player_col_index]
players_series = [matches[s] for s in player_col_index]
players_participation = pd.concat(players_series, axis=0, ignore_index=True).rename('player_id')
players_participation = players_participation.dropna()
players_participation = players_participation.astype('int32')
hist =  players_participation.value_counts().rename('match_count')
players = player_data[['player_api_id', 'player_name']].set_index('player_api_id')
hist = pd.concat([hist, players], axis=1).sort_values(by='match_count', ascending=False)
max_matches_player = hist.iloc[0,hist.columns.get_loc('player_name')]

In [207]:
assert(isinstance(max_matches_player, str))

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [ ]:
foot = {'right': 1, 'left': 0, 1: 1, 0: 0}
players_attributes.loc[:,'preferred_foot'] = players_attributes['preferred_foot'].apply(lambda x: foot[x] if x in foot else np.nan)
corr_attr: pd.DataFrame = players_attributes.loc[:,
                                    ~players_attributes.columns.isin(
                                        ['id', 'date', 'player_fifa_api_id', 'player_api_id', 'date,preferred_foot', 'attacking_work_rate', 'defensive_work_rate'])
                                    ].corr()
n = corr_attr.shape[0]
for i in range(36):
    corr_attr.iloc[i,[False]*i + [True]*(36-i)] = np.nan
col_names = corr_attr.columns
corr_list = pd.concat([corr_attr[x] for x in col_names], keys=col_names)
corr_list = corr_list.loc[~corr_list.isna()].sort_values(ascending=False)
top_correlated_features: pd.Series = corr_list.iloc[0:5]
top_correlated_features = top_correlated_features.index.tolist()
top_correlated_features

In [244]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [ ]:
neymar_similarities = # Your code here

In [ ]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [ ]:
borussia_bundesliga_2008_2009_matches = # Your code here

In [ ]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [ ]:
team_most_matches_bundesliga_2008_2009 = # Your code here

In [ ]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [ ]:
arsenal_won_matches_2015_2016 = # Your code here

In [ ]:
assert(isinstance(arsenal_won_matches_2015_2016, int))

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [ ]:
team_highest_winrate_2015_2016 = # Your code here

In [ ]:
assert(isinstance(team_highest_winrate_2015_2016, str))

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [ ]:
highest_gap_england_2010_2011 = # Your code here

In [ ]:
assert(isinstance(highest_gap_england_2010_2011, int))

### Warning! Do not change anything in the area below

In [ ]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")